# LangSmith en la Era de LangChain 1.0: Una Guía para Principiantes

## Introducción

¿Qué es exactamente LangSmith? ¿Cómo te ayuda? ¿Y cómo ha evolucionado para dar soporte al nuevo ecosistema de LangChain 1.0 y LangGraph 1.0?

Este notebook explicará todo en términos sencillos, con ejemplos claros y analogías.

### Piénsalo de esta manera:

- **LangChain/LangGraph** = Tu coche (la aplicación que construyes)
- **LangSmith** = El cuadro de mandos de tu coche + taller mecánico + circuito de pruebas

Al igual que el cuadro de mandos de un coche te muestra la velocidad, el combustible y el estado del motor, LangSmith te muestra lo que está ocurriendo dentro de tu aplicación LLM. Y al igual que un taller mecánico te ayuda a solucionar problemas, LangSmith te ayuda a depurar y mejorar tus agentes.

¡Vamos a sumergirnos!

---

## Parte 1: ¿Qué es LangSmith?

### La Respuesta Simple

**LangSmith es una plataforma que te ayuda a construir mejores aplicaciones LLM mostrándote lo que está ocurriendo, probando si funciona y facilitando su despliegue.**

### Los Tres Pilares de LangSmith

LangSmith tiene tres superpoderes principales:

1. **👁️ Observabilidad ("Ver qué está pasando")** - Trazado y monitorización
2. **✅ Evaluación ("Probar si funciona")** - Garantía de calidad y pruebas
3. **🚀 Despliegue ("Llevarlo a producción")** - Alojamiento y escalado

Exploremos cada uno.

---

## Parte 2: Pilar 1 - Observabilidad (Ver Qué Está Pasando)

### El Problema

Imagina que has construido un agente que:
1. Busca en la web
2. Lee un documento
3. Genera una respuesta

Pero a veces da respuestas incorrectas. **¿Por qué?**

- ¿La búsqueda devuelve malos resultados?
- ¿El LLM malinterpreta el documento?
- ¿El prompt es confuso?

**Sin LangSmith, estás depurando a ciegas.** 🙈

### La Solución: Trazado

LangSmith **registra cada paso** que da tu agente. Esto se llama **trazado**.

Piensa en ello como una caja negra de un avión: registra todo para que puedas entender qué ocurrió.

### ¿Qué Se Traza?

LangSmith registra:
- 📝 **Cada prompt** enviado al LLM
- 💬 **Cada respuesta** del LLM
- 🔧 **Cada llamada a herramienta** (búsqueda, consulta de base de datos, etc.)
- 📊 **Resultados** de cada herramienta
- ⏱️ **Cuánto tiempo** llevó cada paso
- 💰 **Cuánto costó** (tokens utilizados)
- ❌ **Cualquier error** que ocurrió

### Ejemplo: Traza de Agente Simple

Veamos cómo se ve una traza:

In [ ]:
# Configuración: Habilitar trazado de LangSmith
import os

# Establece estas variables de entorno para habilitar el trazado
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "tu-api-key-aquí"  # Obtén de smith.langchain.com
os.environ["LANGSMITH_PROJECT"] = "mi-primer-proyecto"  # Opcional: organiza las trazas

# ¡Eso es todo! Ahora todo tu código LangChain se trazará automáticamente en LangSmith

In [ ]:
# Ejemplo: Agente simple que busca y responde
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool

# Definir una herramienta de búsqueda simple
@tool
def search_web(query: str) -> str:
    """Buscar información en la web."""
    # Búsqueda simulada
    return f"Resultados de búsqueda para: {query}\nEncontrado: LangChain es un framework para aplicaciones LLM."

# Crear un agente
model = init_chat_model("gpt-4o")
agent = create_agent(model, tools=[search_web])

# Ejecutar el agente - ¡ESTO SE TRAZARÁ AUTOMÁTICAMENTE EN LANGSMITH!
result = agent.invoke({
    "messages": [{"role": "user", "content": "¿Qué es LangChain?"}]
})

print(result["messages"][-1].content)

**Lo que ocurre en LangSmith:**

Cuando ejecutas este código, LangSmith crea una **traza** que muestra:

```
Ejecución: Ejecución del Agente
  ├─ Entrada: "¿Qué es LangChain?"
  ├─ Llamada LLM #1
  │   ├─ Prompt: "Eres un asistente útil... ¿Qué es LangChain?"
  │   ├─ Respuesta: Llamada a herramienta search_web("LangChain")
  │   ├─ Tokens: 150
  │   └─ Duración: 1.2s
  ├─ Llamada a Herramienta: search_web
  │   ├─ Entrada: "LangChain"
  │   ├─ Salida: "LangChain es un framework..."
  │   └─ Duración: 0.3s
  ├─ Llamada LLM #2
  │   ├─ Prompt: "Eres un asistente útil... [resultados de búsqueda]"
  │   ├─ Respuesta: "LangChain es un framework para construir aplicaciones LLM..."
  │   ├─ Tokens: 200
  │   └─ Duración: 1.5s
  └─ Salida: Respuesta final
  
Duración Total: 3.0s
Coste Total: $0.002
```

**¡Puedes verlo TODO!** Esto es increíblemente potente para la depuración.

### Avanzado: Trazado Selectivo

A veces no quieres trazar TODO (especialmente en producción). LangSmith te permite ser selectivo:

In [ ]:
from langchain.tracing import tracing_context

# Solo trazar llamadas específicas
with tracing_context(enabled=True, project_name="depuracion-produccion"):
    result = agent.invoke({"messages": [{"role": "user", "content": "Consulta de prueba"}]})
    # Esto se trazará

# Esto no se trazará
result = agent.invoke({"messages": [{"role": "user", "content": "Otra consulta"}]})

### Metadatos y Etiquetas

Puedes añadir información personalizada a las trazas:

In [ ]:
# Añadir metadatos para ayudar a filtrar y analizar trazas
result = agent.invoke(
    {"messages": [{"role": "user", "content": "¿Qué es LangChain?"}]},
    config={
        "metadata": {
            "user_id": "usuario123",
            "session_id": "sesion456",
            "environment": "produccion"
        },
        "tags": ["soporte-cliente", "usuario-premium"]
    }
)

# ¡Ahora puedes filtrar trazas por usuario, sesión o etiqueta en LangSmith!

### Beneficios Clave del Trazado

1. **🐛 Depurar más rápido** - Ver exactamente dónde van mal las cosas
2. **⚡ Optimizar rendimiento** - Encontrar pasos lentos
3. **💰 Controlar costes** - Ver qué llamadas LLM son caras
4. **📊 Entender comportamiento** - Ver cómo tu agente toma decisiones
5. **🔍 Monitorizar producción** - Detectar problemas antes de que se quejen los usuarios

---

## Parte 3: LangSmith Studio (ahora el equipo de LangChain está proponiendo Agent Builder como mejor alternativa)

### ¿Qué es Studio?

**LangSmith Studio es una interfaz visual GRATUITA para desarrollar y probar tus agentes localmente.**

Piensa en ello como:
- Un **área de pruebas** donde puedes probar tu agente
- Un **depurador** que te muestra cada paso en tiempo real
- Una **máquina del tiempo** que te permite reproducir conversaciones

### Características Clave

1. **👁️ Visualización en tiempo real** - Ve la ejecución de tu agente mientras ocurre
2. **🔄 Recarga en caliente** - Cambia tu código y ve actualizaciones al instante
3. **⏮️ Reproducción de hilos** - Vuelve a ejecutar conversaciones desde cualquier punto
4. **🔬 Inspección paso a paso** - Examina cada prompt, llamada a herramienta y respuesta
5. **📊 Visualización de métricas** - Ve recuentos de tokens, latencia y costes

### Configurar Studio

In [ ]:
# 1. Instalar la CLI de LangGraph
# Ejecutar en terminal:
# pip install langgraph-cli

# 2. Crear un archivo de configuración langgraph.json
# En tu directorio de proyecto, crea: langgraph.json
'''
{
  "dependencies": ["."],
  "graphs": {
    "mi_agente": "./mi_agente.py:agent"
  },
  "env": ".env"
}
'''

# 3. Ejecutar el servidor de desarrollo
# En terminal:
# langgraph dev

# 4. Abrir la UI en http://localhost:8123
# ¡Ahora tienes una interfaz visual para probar tu agente!

### Usar Studio

Una vez que Studio esté ejecutándose, puedes:

```
┌─────────────────────────────────────────┐
│  LangSmith Studio                       │
├─────────────────────────────────────────┤
│                                         │
│  Entrada: "¿Qué es LangChain?"         │
│                                         │
│  ┌───────────────────────────────────┐ │
│  │ Paso 1: Decisión del Agente       │ │
│  │ Prompt: "Eres un asistente útil..."│ │
│  │ Respuesta: Llamar herramienta     │ │
│  │ Tokens: 150 | Tiempo: 1.2s        │ │
│  └───────────────────────────────────┘ │
│                                         │
│  ┌───────────────────────────────────┐ │
│  │ Paso 2: Ejecución de Herramienta  │ │
│  │ Herramienta: search_web           │ │
│  │ Entrada: "LangChain"              │ │
│  │ Salida: "LangChain es..."         │ │
│  │ Tiempo: 0.3s                      │ │
│  └───────────────────────────────────┘ │
│                                         │
│  ┌───────────────────────────────────┐ │
│  │ Paso 3: Respuesta Final           │ │
│  │ Prompt: "Basado en búsqueda..."   │ │
│  │ Respuesta: "LangChain es un..."   │ │
│  │ Tokens: 200 | Tiempo: 1.5s        │ │
│  └───────────────────────────────────┘ │
│                                         │
│  Tiempo Total: 3.0s | Coste: $0.002    │
└─────────────────────────────────────────┘
```

**Puedes hacer clic en cada paso para ver:**
- El prompt exacto enviado al LLM
- La respuesta completa
- Todos los metadatos
- Cualquier error

**Y puedes:**
- Volver a ejecutar desde cualquier paso
- Probar diferentes entradas
- Comparar múltiples ejecuciones

### Por Qué Studio es Increíble para Principiantes

Cuando estás aprendiendo, Studio te ayuda a:

1. **Entender cómo funcionan los agentes** - Ver el proceso de toma de decisiones visualmente
2. **Depurar sin frustración** - ¡No más depuración con `print()`!
3. **Experimentar rápidamente** - Probar diferentes prompts y ver resultados inmediatamente
4. **Aprender de los errores** - Ver exactamente qué salió mal

**¡Es como tener un profesor que te muestra paso a paso cómo funciona tu código!**

---

## Parte 4: Pilar 2 - Evaluación (Probar Si Funciona)

### El Problema

Has construido un agente. Funciona con tu entrada de prueba. **Pero:**

- ¿Funciona con 100 entradas diferentes?
- ¿Es mejor que la versión 1?
- ¿Funcionará para tus usuarios?

**Probar manualmente es lento y poco fiable.** Necesitas evaluación automatizada.

### La Solución: Conjuntos de Datos y Evaluadores

LangSmith te ayuda a:
1. Crear **conjuntos de datos de prueba** (colecciones de entradas y salidas esperadas)
2. Ejecutar tu agente en todo el conjunto de datos
3. **Evaluar** automáticamente la calidad de las respuestas
4. Seguir mejoras a lo largo del tiempo

### Crear un Conjunto de Datos

In [ ]:
from langsmith import Client

client = Client()

# Crear un conjunto de datos de casos de prueba
dataset = client.create_dataset(
    dataset_name="prueba-qa-langchain",
    description="Casos de prueba para agente de Q&A de LangChain"
)

# Añadir ejemplos
examples = [
    {
        "input": {"question": "¿Qué es LangChain?"},
        "expected_output": "LangChain es un framework para construir aplicaciones LLM."
    },
    {
        "input": {"question": "¿Qué es LangGraph?"},
        "expected_output": "LangGraph es una biblioteca para construir aplicaciones multi-agente con estado."
    },
    {
        "input": {"question": "¿Cómo instalo LangChain?"},
        "expected_output": "Puedes instalar LangChain usando: pip install langchain"
    },
]

for example in examples:
    client.create_example(
        inputs=example["input"],
        outputs={"answer": example["expected_output"]},
        dataset_id=dataset.id
    )

### Ejecutar Evaluaciones

In [ ]:
from langsmith.evaluation import evaluate

# Definir cómo ejecutar tu agente
def run_agent(inputs):
    result = agent.invoke({
        "messages": [{"role": "user", "content": inputs["question"]}]
    })
    return {"answer": result["messages"][-1].content}

# Definir cómo evaluar respuestas
def evaluate_answer(run, example):
    """Comprobar si la respuesta es correcta."""
    predicted = run.outputs["answer"]
    expected = example.outputs["answer"]
    
    # Comprobación simple: ¿la respuesta contiene información clave?
    score = 1.0 if expected.lower() in predicted.lower() else 0.0
    
    return {"key": "correctness", "score": score}

# Ejecutar evaluación en todo el conjunto de datos
results = evaluate(
    run_agent,
    data="prueba-qa-langchain",
    evaluators=[evaluate_answer],
    experiment_prefix="agente-v1"
)

# Los resultados muestran:
# - Cuántos pasaron/fallaron
# - Puntuación media
# - Resultados detallados para cada ejemplo

### Avanzado: LLM como Juez

En lugar de escribir reglas, puedes usar un LLM para evaluar respuestas:

In [ ]:
from langchain.evaluation import load_evaluator

# Usar un LLM para juzgar si la respuesta es útil
evaluator = load_evaluator(
    "labeled_criteria",
    criteria="helpfulness",
    llm=model
)

def llm_evaluate(run, example):
    result = evaluator.evaluate_strings(
        input=example.inputs["question"],
        prediction=run.outputs["answer"],
        reference=example.outputs["answer"]
    )
    return {
        "key": "helpfulness",
        "score": result["score"],
        "comment": result["reasoning"]
    }

# ¡Ahora el LLM juzga si las respuestas son útiles!

### Comparar Versiones

LangSmith te permite comparar diferentes versiones de tu agente:

```
Versión 1 (prompt antiguo):
  Corrección: 70%
  Utilidad: 3.5/5
  Tiempo de Respuesta Medio: 2.5s

Versión 2 (prompt nuevo):
  Corrección: 85%  ✅ ¡Mejorado!
  Utilidad: 4.2/5  ✅ ¡Mejorado!
  Tiempo de Respuesta Medio: 2.1s  ✅ ¡Más rápido!
```

Esto te ayuda a tomar **decisiones basadas en datos** sobre qué versión es mejor.

### Beneficios Clave de la Evaluación

1. **🎯 Detectar regresiones** - Asegurarse de que las actualizaciones no rompen nada
2. **📈 Seguir mejoras** - Ver si tus cambios realmente ayudan
3. **🏆 Comparar enfoques** - Probar diferentes prompts, modelos o arquitecturas
4. **✅ Garantía de calidad** - Asegurar calidad consistente antes del despliegue
5. **💡 Aprender patrones** - Entender qué funciona y qué no

---

## Parte 5: Pilar 3 - Despliegue (Llevarlo a Producción)

### El Problema

Tu agente funciona genial en tu portátil. Ahora necesitas:
- Hacerlo disponible 24/7
- Manejar muchos usuarios a la vez
- Mantenerlo rápido y fiable
- Monitorizarlo en producción

**Las plataformas de alojamiento tradicionales no están construidas para agentes con estado.** Están diseñadas para aplicaciones web simples.

### La Solución: Despliegues de LangSmith

LangSmith proporciona **alojamiento gestionado específicamente diseñado para agentes LangGraph**.

### ¿Por Qué Alojamiento Especial para Agentes?

Alojamiento web tradicional:
```
Petición → Procesar → Respuesta
(De corta duración, sin estado)
```

Agentes LangGraph:
```
Petición → Pensar → Llamar Herramienta → Esperar → Pensar → Llamar Herramienta → Respuesta
(De larga duración, con estado)
```

LangSmith maneja:
- **Persistencia de estado** - Recordar historial de conversación
- **Ejecución en segundo plano** - Los agentes pueden tardar minutos en completarse
- **Escalado** - Manejar muchos usuarios concurrentes
- **Streaming** - Enviar resultados parciales según se generan

### Desplegar Tu Agente

In [ ]:
# Proceso de Despliegue (Hecho a través de la UI de LangSmith):

# 1. Subir tu código a GitHub
# Tu repositorio debe tener:
# - Tu código del agente
# - Configuración langgraph.json
# - requirements.txt con dependencias

# 2. Conectar GitHub a LangSmith
# Ir a smith.langchain.com → Despliegues → Conectar GitHub

# 3. Seleccionar tu repositorio y desplegar
# LangSmith:
# - Clonará tu repositorio
# - Instalará dependencias
# - Construirá tu agente
# - Desplegará en infraestructura
# (Tarda ~15 minutos)

# 4. Obtener tu URL de despliegue
# https://tu-agente.langsmith.app

### Usar Tu Agente Desplegado

In [ ]:
# Opción 1: Usar el SDK de Python
from langgraph_sdk import get_client

client = get_client(url="https://tu-agente.langsmith.app")

# Crear un hilo (conversación)
thread = await client.threads.create()

# Enviar un mensaje
response = await client.runs.create(
    thread["thread_id"],
    assistant_id="mi_agente",
    input={"messages": [{"role": "user", "content": "¿Qué es LangChain?"}]}
)

# Transmitir la respuesta
async for chunk in client.runs.stream(
    thread["thread_id"],
    response["run_id"]
):
    print(chunk)

In [ ]:
# Opción 2: Usar REST API (desde cualquier lenguaje)
import requests

url = "https://tu-agente.langsmith.app/threads"

# Crear hilo
response = requests.post(url)
thread_id = response.json()["thread_id"]

# Enviar mensaje
response = requests.post(
    f"{url}/{thread_id}/runs",
    json={
        "assistant_id": "mi_agente",
        "input": {
            "messages": [{"role": "user", "content": "¿Qué es LangChain?"}]
        }
    }
)

print(response.json())

### Monitorización de Producción

Una vez desplegado, LangSmith automáticamente:
- ✅ Traza todas las peticiones de producción
- ✅ Monitoriza rendimiento y errores
- ✅ Sigue los costes
- ✅ Muestra análisis de uso

Puedes:
- Ver qué usuarios están teniendo problemas
- Encontrar peticiones lentas o caras
- Configurar alertas para errores
- Volver a versiones anteriores

### Beneficios Clave del Despliegue

1. **🚀 Despliegue rápido** - De GitHub a producción en 15 minutos
2. **📈 Auto-escalado** - Maneja picos de tráfico automáticamente
3. **💪 Construido para agentes** - Soporta flujos de trabajo de larga duración y con estado
4. **📊 Monitorización incluida** - Trazas y métricas listas para usar
5. **🔧 Actualizaciones fáciles** - Subir a GitHub, redespliegue automático

---

## Parte 6: Cómo Ha Evolucionado LangSmith

### Los Viejos Tiempos (Pre-2024)

**LangSmith 0.x:**
- Principalmente una **herramienta de trazado**
- Configuración manual requerida
- Características de evaluación limitadas
- Sin capacidades de despliegue
- UI básica

**Flujo de trabajo de desarrollo:**
```
Código → Probar localmente → Desplegar en plataforma separada → Esperar que funcione
```

### La Nueva Era (LangChain 1.0 y LangGraph 1.0)

**LangSmith en 2024-2025:**
- **Plataforma completa** para todo el ciclo de vida de desarrollo
- Trazado automático para `create_agent`
- Framework de evaluación rico
- Despliegue integrado
- Studio para desarrollo local
- Monitorización de producción

**Nuevo flujo de trabajo:**
```
Desarrollar en Studio → Evaluar con conjuntos de datos → Desplegar con un clic → Monitorizar en producción
```

¡Todo en una plataforma!

### Hitos Clave de la Evolución

#### 1. Lanzamiento de Studio (2024)
- **Antes:** Probar agentes ejecutando código repetidamente
- **Después:** Interfaz visual con recarga en caliente y reproducción

#### 2. Trazado Automático para `create_agent`
- **Antes:** Instrumentación manual requerida
- **Después:** Solo establecer variables de entorno, el trazado funciona automáticamente

#### 3. Framework de Evaluación
- **Antes:** Escribir scripts de prueba personalizados
- **Después:** Conjuntos de datos integrados, evaluadores y herramientas de comparación

#### 4. Despliegues de LangSmith
- **Antes:** Desplegar a AWS/GCP/Azure manualmente
- **Después:** Desplegar desde GitHub en 15 minutos

#### 5. Trazado de Subgrafos
- **Antes:** Solo ver trazas de agente de nivel superior
- **Después:** Ver dentro de subgrafos anidados y sistemas multi-agente

#### 6. Gestión de Hilos
- **Antes:** Gestionar estado manualmente
- **Después:** Gestión de hilos y puntos de control integrada

### Integración con LangChain 1.0

LangSmith ahora está **profundamente integrado** con LangChain 1.0:

```python
# LangChain 0.x
from langchain.callbacks import LangChainTracer
tracer = LangChainTracer()
# ... configuración manual ...

# LangChain 1.0
os.environ["LANGSMITH_TRACING"] = "true"
# ¡Eso es todo! Todo se traza automáticamente
```

¡No más callbacks, no más instrumentación manual!

### Integración con LangGraph 1.0

LangGraph 1.0 está **diseñado para LangSmith**:

- **Puntos de control** - LangSmith almacena el estado de la conversación
- **Visibilidad de subgrafos** - Ver cada nivel de sistemas multi-agente
- **Humano en el bucle** - Interrupción y reanudación integradas
- **Soporte de Studio** - Visualizar grafos complejos
- **Optimizado para despliegue** - Las aplicaciones LangGraph se despliegan sin problemas

```python
# LangGraph funciona automáticamente con LangSmith
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver

graph = StateGraph(...)
# ... construir grafo ...
graph = graph.compile(checkpointer=MemorySaver())

# Transmitir con visibilidad de subgrafos
for chunk in graph.stream(input, subgraphs=True):
    print(chunk)
    # ¡Todo esto se traza en LangSmith!
```

---

## Parte 7: Ejemplo de Flujo de Trabajo Completo

Veamos cómo todo se une en un proyecto real:

### Fase 1: Desarrollo (Usando Studio)

```bash
# Terminal
langgraph dev
# Abre Studio en http://localhost:8123
```

En Studio:
1. Probar diferentes prompts visualmente
2. Ver qué funciona y qué no
3. Corregir errores viendo la ejecución exacta
4. Recargar cambios al instante

### Fase 2: Evaluación (Crear Suite de Pruebas)

In [ ]:
# Crear conjunto de datos de prueba
from langsmith import Client

client = Client()
dataset = client.create_dataset("suite-prueba-produccion")

# Añadir 100 casos de prueba
test_cases = [
    {"question": "¿Qué es LangChain?", "expected": "..."},
    {"question": "¿Cómo uso herramientas?", "expected": "..."},
    # ... 98 más ...
]

for case in test_cases:
    client.create_example(
        inputs={"question": case["question"]},
        outputs={"answer": case["expected"]},
        dataset_id=dataset.id
    )

In [ ]:
# Ejecutar evaluación
from langsmith.evaluation import evaluate

results = evaluate(
    lambda x: agent.invoke({"messages": [{"role": "user", "content": x["question"]}]}),
    data="suite-prueba-produccion",
    evaluators=[correctness_evaluator, helpfulness_evaluator],
    experiment_prefix="pre-despliegue"
)

print(f"Corrección: {results['correctness']}%")
print(f"Utilidad: {results['helpfulness']}/5")

# ¡Solo desplegar si las puntuaciones son buenas!

### Fase 3: Despliegue

In [ ]:
# Subir a GitHub
# git push origin main

# En la UI de LangSmith:
# 1. Ir a Despliegues
# 2. Seleccionar repositorio
# 3. Clic en Desplegar
# 4. Esperar 15 minutos
# 5. Obtener URL de despliegue

# ¡Ahora tu agente está en vivo!

### Fase 4: Monitorización de Producción

In [ ]:
# Todas las peticiones de producción se trazan automáticamente
# Ir a la UI de LangSmith para ver:

# Panel:
# - Total de peticiones: 10,000
# - Tasa de éxito: 98%
# - Latencia media: 2.3s
# - Coste total: $45.67

# Errores:
# - 200 peticiones fallaron
# - Clic para ver trazas
# - Identificar el problema
# - Corregir y redesplegar

# Feedback de usuarios:
# - 95% positivo
# - Ver problemas específicos
# - Mejorar basándose en feedback

### Fase 5: Mejora Continua

In [ ]:
# Usar trazas de producción para mejorar

# 1. Encontrar trazas problemáticas
# UI de LangSmith: Filtrar por "Estado: Error" o "Duración > 10s"

# 2. Añadirlas al conjunto de datos de prueba
client.create_examples_from_traces(
    trace_ids=["trace-1", "trace-2", "trace-3"],
    dataset_id=dataset.id
)

# 3. Corregir los problemas
# Actualizar prompts, añadir manejo de errores, etc.

# 4. Volver a evaluar
results = evaluate(..., experiment_prefix="v2")

# 5. Comparar versiones
# UI de LangSmith: Comparar v1 vs v2
# Ver: ¡v2 es 10% mejor!

# 6. Desplegar v2
# git push origin main

# 7. ¡Repetir!

---

## Parte 8: Precios y Planes de LangSmith

### Nivel Gratuito (¡Genial para Aprender!)

**Incluye:**
- ✅ Studio (desarrollo local ilimitado)
- ✅ Trazado (hasta 5,000 trazas/mes)
- ✅ Evaluación básica
- ✅ Conjuntos de datos pequeños

**Perfecto para:**
- Aprender LangChain y LangGraph
- Proyectos personales
- Prototipos
- Pequeños proyectos paralelos

### Niveles de Pago (Para Producción)

**Desarrollador ($39/mes):**
- Más trazas
- Conjuntos de datos más grandes
- Evaluaciones avanzadas
- Características de equipo

**Equipo ($199/mes):**
- Trazas ilimitadas
- Despliegues incluidos
- Soporte prioritario
- SSO y características de seguridad

**Empresa (Precio personalizado):**
- Opción auto-hospedada
- SLA personalizados
- Soporte dedicado
- Cumplimiento avanzado

---

## Parte 9: Comenzar - Paso a Paso

### Paso 1: Registrarse (Gratis)

1. Ir a [smith.langchain.com](https://smith.langchain.com)
2. Clic en "Sign Up"
3. Crear una cuenta (¡gratis!)
4. Obtener tu API key

### Paso 2: Habilitar Trazado

In [ ]:
# Añadir a tu código o archivo .env
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "lsv2_pt_..."  # De smith.langchain.com
os.environ["LANGSMITH_PROJECT"] = "mi-primer-proyecto"

# ¡Eso es todo! Ahora ejecuta tu código LangChain

### Paso 3: Ejecutar Algo de Código

In [ ]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o")
agent = create_agent(model, tools=[])

# ¡Esto se trazará!
result = agent.invoke({
    "messages": [{"role": "user", "content": "¡Hola!"}]
})

print(result["messages"][-1].content)

### Paso 4: Ver Tu Primera Traza

1. Ir a [smith.langchain.com](https://smith.langchain.com)
2. Clic en tu proyecto
3. ¡Ver tu traza!
4. Clic para explorar

**Verás:**
- El prompt exacto
- La respuesta del modelo
- Cuánto tiempo tardó
- Cuánto costó
- Todos los metadatos

### Paso 5: Configurar Studio (Opcional pero Recomendado)

In [ ]:
# Instalar CLI
# pip install langgraph-cli

# Crear archivo de configuración: langgraph.json
'''
{
  "dependencies": ["."],
  "graphs": {
    "mi_agente": "./agent.py:agent"
  },
  "env": ".env"
}
'''

# Ejecutar
# langgraph dev

# Abrir http://localhost:8123

### Paso 6: ¡Experimentar!

Ahora tienes todo configurado. Prueba:

1. **Construir un agente simple**
2. **Ver trazas** en LangSmith
3. **Probar en Studio**
4. **Crear un conjunto de datos de prueba**
5. **Ejecutar evaluaciones**
6. **Mejorar basándose en resultados**

¡Estás aprendiendo haciendo!

---

## Parte 10: Preguntas Comunes

### P: ¿Necesito LangSmith para usar LangChain?

**R:** ¡No! LangChain funciona bien sin LangSmith. Pero LangSmith hace el desarrollo mucho más fácil mostrándote lo que está pasando dentro de tus agentes.

### P: ¿LangSmith es gratis?

**R:** ¡Sí! El nivel gratuito es generoso (5,000 trazas/mes, uso ilimitado de Studio). Perfecto para aprender y proyectos pequeños.

### P: ¿Puedo usar LangSmith con otros frameworks?

**R:** ¡Sí! LangSmith funciona con cualquier código Python que llame a LLMs. Puedes instrumentar manualmente código que no sea LangChain.

### P: ¿LangSmith almacena mis datos?

**R:** Sí, las trazas se almacenan en LangSmith. Pero puedes:
- Controlar qué se traza
- Auto-hospedar LangSmith
- Usar políticas de retención de datos
- LangSmith cumple con SOC 2 Type 2, GDPR y HIPAA

### P: ¿Cómo depuro sin LangSmith?

**R:** Puedes usar sentencias `print()`, pero es mucho más difícil. LangSmith te muestra la imagen completa con visualizaciones bonitas.

### P: ¿Puedo desplegar sin Despliegues de LangSmith?

**R:** ¡Sí! Puedes desplegar en AWS, GCP, Azure o cualquier otra plataforma. Pero Despliegues de LangSmith lo hace mucho más fácil.

### P: ¿En qué se diferencia Studio de los notebooks Jupyter?

**R:** Jupyter es para ejecutar código. Studio es específicamente para probar agentes visualmente. Muestra el flujo de ejecución, te permite reproducir hilos y se actualiza cuando cambias el código.

### P: ¿Puedo usar LangSmith para monitorización de producción?

**R:** ¡Sí! Es uno de sus usos principales. Todas las trazas de producción se capturan automáticamente, y puedes configurar alertas, paneles y análisis.

---

## Parte 11: Mejores Prácticas

### Para Desarrollo

1. **Usar siempre Studio** - Ver la ejecución de tu agente visualmente
2. **Usar nombres de proyecto descriptivos** - `mi-chatbot-v1` no `prueba`
3. **Añadir metadatos a trazas** - IDs de usuario, IDs de sesión ayudan a depurar
4. **Etiquetar tus trazas** - Hace el filtrado más fácil
5. **Revisar trazas regularmente** - Aprender del comportamiento de tu agente

### Para Pruebas

1. **Empezar con un conjunto de datos pequeño** - 10-20 ejemplos para comenzar
2. **Añadir casos fallidos** - Cada bug debería convertirse en una prueba
3. **Usar múltiples evaluadores** - Corrección, utilidad, seguridad, etc.
4. **Comparar versiones** - Siempre hacer pruebas A/B de mejoras
5. **Automatizar evaluación** - Ejecutar pruebas antes del despliegue

### Para Producción

1. **Muestrear trazas en producción** - No trazar el 100% (usar 10-20%)
2. **Monitorizar métricas clave** - Tasa de éxito, latencia, coste
3. **Configurar alertas** - Ser notificado de errores
4. **Usar trazas de producción para mejora** - Añadir casos límite a conjuntos de datos
5. **Versionar tus prompts** - Seguir qué cambió cuándo

### Para Equipos

1. **Usar nomenclatura de proyecto consistente** - `nombre-equipo/nombre-agente/version`
2. **Documentar evaluadores** - Explicar qué verifica cada uno
3. **Compartir trazas interesantes** - LangSmith tiene enlaces para compartir
4. **Revisar trazas juntos** - Genial para aprender
5. **Establecer estándares de calidad** - "Debe pasar el 90% de las pruebas antes de desplegar"

---

## Conclusión

### Lo Que Hemos Aprendido

LangSmith es una **plataforma completa** para construir aplicaciones LLM:

1. **👁️ Observabilidad** - Ver qué está pasando con trazado automático
2. **✅ Evaluación** - Probar si funciona con conjuntos de datos y evaluadores
3. **🚀 Despliegue** - Llevar a producción en 15 minutos

### Cómo Ha Evolucionado

**Antes:** Solo una herramienta de trazado
**Ahora:** Plataforma completa de desarrollo, pruebas y despliegue

**Antes:** Configuración manual requerida
**Ahora:** Integración automática con LangChain 1.0 y LangGraph 1.0

**Antes:** Limitado a observabilidad
**Ahora:** Studio, evaluación, despliegue, monitorización - ¡todo!

### ¿Por Qué Usar LangSmith?

**Para Aprender:**
- Ver cómo funcionan realmente los agentes
- Depurar problemas visualmente
- Aprender de los errores

**Para Construir:**
- Desarrollar más rápido con Studio
- Probar exhaustivamente con evaluación
- Desplegar fácilmente con un clic

**Para Producción:**
- Monitorizar todo
- Detectar problemas temprano
- Mejorar continuamente

### Comenzar

1. Registrarse en [smith.langchain.com](https://smith.langchain.com) (¡gratis!)
2. Establecer variables de entorno
3. Ejecutar tu código LangChain
4. Ver trazas en la UI
5. Instalar Studio para desarrollo local

¡Eso es todo! Estás listo para construir mejores aplicaciones LLM con LangSmith.

¡Feliz construcción! 🚀

---

## Recursos Adicionales

### Documentación Oficial
- [Documentación de LangSmith](https://docs.langchain.com/langsmith)
- [Guía de Observabilidad de LangChain](https://docs.langchain.com/oss/python/langchain/observability)
- [Guía de LangChain Studio](https://docs.langchain.com/oss/python/langchain/studio)
- [Guía de Despliegue de LangSmith](https://docs.langchain.com/oss/python/langchain/deploy)


Recuerda: La mejor manera de aprender es haciendo. ¡Construye algo, trázalo, pruébalo, despliégalo!